In [1]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline
import random
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm


In [3]:
df = pd.read_csv("data/songListCleaned.csv")
features = ["popularity", "duration_ms", "danceability", "energy", "loudness", "speechiness",
            "acousticness", "instrumentalness", "liveness", "valence", "tempo"]
# labels = {
#     # Valence
#     "high valence": [0.7, 1.0],
#     "medium valence": [0.3, 0.7],
#     "low valence": [0.0, 0.3],

#     # Popularity
#     "high popularity": [0.7, 1.0],
#     "medium popularity": [0.3, 0.7],
#     "low popularity": [0.0, 0.3],

#     # Duration (normalized to [0.0, 1.0])
#     "high duration": [0.7, 1.0],
#     "medium duration": [0.3, 0.7],
#     "short duration": [0.0, 0.3],

#     # Danceability
#     "high danceability": [0.7, 1.0],
#     "medium danceability": [0.3, 0.7],
#     "low danceability": [0.0, 0.3],

#     # Energy
#     "high energy": [0.7, 1.0],
#     "medium energy": [0.3, 0.7],
#     "low energy": [0.0, 0.3],

#     # Loudness (normalized to [0.0, 1.0])
#     "high loudness": [0.7, 1.0],
#     "medium loudness": [0.3, 0.7],
#     "low loudness": [0.0, 0.3],

#     # Speechiness
#     "high speechiness": [0.7, 1.0],
#     "medium speechiness": [0.3, 0.7],
#     "low speechiness": [0.0, 0.3],

#     # Acousticness
#     "high acousticness": [0.7, 1.0],
#     "medium acousticness": [0.3, 0.7],
#     "low acousticness": [0.0, 0.3],

#     # Instrumentalness
#     "high instrumentalness": [0.7, 1.0],
#     "medium instrumentalness": [0.3, 0.7],
#     "low instrumentalness": [0.0, 0.3],

#     # Liveness
#     "high liveness": [0.7, 1.0],
#     "medium liveness": [0.3, 0.7],
#     "low liveness": [0.0, 0.3],

#     # Tempo (normalized to [0.0, 1.0])
#     "high tempo": [0.7, 1.0],
#     "medium tempo": [0.3, 0.7],
#     "low tempo": [0.0, 0.3],
# }

In [4]:
# # Define min and max values for each feature (adjust based on your dataset)
# feature_ranges = {
#     "valence": (0.0, 1.0),
#     "popularity": (0, 100),
#     "duration_ms": (8586, 5237295),  # Adjust based on your dataset
#     "danceability": (0.0, 0.985),
#     "energy": (0.0, 1.0),
#     "loudness": (-49.531, 4.532),  # Loudness ranges from -60 dB to 0 dB
#     "speechiness": (0.0, 0.965),
#     "acousticness": (0.0, 0.996),
#     "instrumentalness": (0.0, 1.0),
#     "liveness": (0.0, 1.0),
#     "tempo": (0, 243.372),  # Tempo ranges from 0 BPM to 250 BPM
# }

# # Normalize features to [0.0, 1.0]
# for feature, (min_val, max_val) in feature_ranges.items():
#     df[f"{feature}_normalized"] = (df[feature] - min_val) / (max_val - min_val)

# # Display the updated dataset
# print(df.head())

In [5]:
#remove trash columns
df = df.drop(['Unnamed: 0.1', 'artists', 'album_name', 'track_name', 'explicit', 'key',
        'mode', 'time_signature'], axis=1)

In [29]:
df.head(10)

,Unnamed: 0,track_id,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,39463,0000vdREvCVMxbQTkS888c,44,160725,0.910,0.374,-9.844,0.1990,0.075700,0.003010,0.1540,0.432,104.042,german
1,17586,000CC8EParg64OmTxVnZ0p,47,322933,0.269,0.516,-7.361,0.0366,0.406000,0.000000,0.1170,0.341,178.174,club
2,73441,000Iz0K615UepwSJ5z2RE5,22,515360,0.686,0.560,-13.264,0.0462,0.001140,0.181000,0.1110,0.108,119.997,minimal-techno
3,51365,000RDCYioLteXcutOjeweY,62,190203,0.679,0.770,-3.537,0.1900,0.058300,0.000000,0.0825,0.839,161.721,hip-hop
4,73649,000qpdoc97IMTBvF8gwcpy,19,331240,0.519,0.431,-13.606,0.0291,0.000964,0.720000,0.0916,0.234,129.971,minimal-techno
5,18342,0017XiMkqbTfF2AUOzlhj6,24,127040,0.536,0.780,-9.449,0.9450,0.792000,0.000000,0.7350,0.452,173.912,comedy
6,15028,001APMDOl3qtx1526T11n1,0,176320,0.613,0.471,-6.644,0.1070,0.316000,0.000001,0.1170,0.406,143.064,"chill,soul"
7,100420,001YQlnDSduXd5LgBd66gT,38,177266,0.554,0.921,-4.589,0.0758,0.019400,0.088100,0.3290,0.700,183.571,"ska,punk-rock"
8,7118,001pyq8FLNSL1C8orNLI0b,30,214600,0.580,0.290,-11.942,0.0272,0.261000,0.000000,0.1250,0.497,91.321,bluegrass
9,46690,002qpSULhHAw6DGqFxbaO1,17,410666,0.531,0.950,-9.744,0.0433,0.001220,0.826000,0.0613,0.553,159.974,happy


In [39]:
for ind, row in df.head(10).iterrows():
    print(row["track_genre"].split(","))

['german']
['club']
['minimal-techno']
['hip-hop']
['minimal-techno']
['comedy']
['chill', 'soul']
['ska', 'punk-rock']
['bluegrass']
['happy']


In [6]:
labels = {}

# Loop through the specified columns
print(len(df.columns))
for col in df.columns[2:13]:
    # Calculate mean and standard deviation
    mn = df[col].mean()
    std = df[col].std()
    min = df[col].min()
    max = df[col].max()
    quantiles = norm.ppf([0.33333, 0.66666], loc=mn, scale=std)

    print(quantiles)

    # Generate random samples from the normal distribution
    samples = norm.rvs(loc=mn, scale=std, size=len(df))
    
    labels["high " + col] = [quantiles[1], max]
    labels["medium " + col] = [quantiles[0], quantiles[1]]
    labels["low " + col] = [min, quantiles[0]]

print(labels)

14
[24.33680521 42.06213859]
[180494.49920464 277791.12563666]
[0.48605866 0.63826919]
[0.52392875 0.74498114]
[-10.74809182  -6.25003931]
[0.03864951 0.13623205]
[0.18255797 0.47400267]
[0.03392167 0.31290016]
[0.13302708 0.30090977]
[0.35624953 0.58269212]
[109.08536386 135.03007629]
{'high popularity': [42.06213858625163, 100], 'medium popularity': [24.336805205185797, 42.06213858625163], 'low popularity': [0, 24.336805205185797], 'high duration_ms': [277791.1256366565, 5237295], 'medium duration_ms': [180494.499204635, 277791.1256366565], 'low duration_ms': [8586, 180494.499204635], 'high danceability': [0.6382691939772152, 0.985], 'medium danceability': [0.48605865656399494, 0.6382691939772152], 'low danceability': [0.0, 0.48605865656399494], 'high energy': [0.7449811392830035, 1.0], 'medium energy': [0.5239287468845211, 0.7449811392830035], 'low energy': [0.0, 0.5239287468845211], 'high loudness': [-6.2500393083862695, 4.532], 'medium loudness': [-10.748091817015506, -6.250039308

In [7]:
# modified_labels = {}
# # Loop through the specified columns
# for col in df.columns[22:]:
#     # Calculate mean and standard deviation
#     mn = df[col].mean()
#     std = df[col].std()
#     min = df[col].min()
#     max = df[col].max()
#     quantiles = norm.ppf([0.33333, 0.66666], loc=mn, scale=std)

#     print(quantiles)

#     # Generate random samples from the normal distribution
#     samples = norm.rvs(loc=mn, scale=std, size=len(df))
    
#     modified_labels["high " + col] = [mn + quantiles[1], max]
#     modified_labels["medium " + col] = [quantiles[0], quantiles[1]]
#     modified_labels["low " + col] = [min, quantiles[0]]
    
#     # Create a figure and axis for the plot
#     fig, ax = plt.subplots(figsize=(8, 6))

#     # Plot the histogram of the generated samples
#     ax.hist(samples, bins=30, density=True, alpha=0.6, color='blue', label='Generated Samples')

#     # Plot the theoretical PDF
#     x = np.linspace(mn - 3 * std, mn + 3 * std, 1000)  # Range for the PDF
#     pdf = norm.pdf(x, loc=mn, scale=std)  # Calculate the PDF
#     ax.plot(x, pdf, 'r-', lw=2, label='Theoretical PDF')

#     # Add labels and title
#     ax.set_xlabel(col)
#     ax.set_ylabel('Density')
#     ax.set_title(f'Distribution of {col} (Mean = {mn:.2f}, Std = {std:.2f})')
#     ax.legend()

#     # Show the plot
#     plt.show()

# print(modified_labels.keys())

In [27]:
for index, rows in df.iterrows():
    curr = rows["track_genre"].split(",")
    for gen in curr:
        if gen not in labels:
            labels[gen] = None

print(labels)

{'high popularity': [42.06213858625163, 100], 'medium popularity': [24.336805205185797, 42.06213858625163], 'low popularity': [0, 24.336805205185797], 'high duration_ms': [277791.1256366565, 5237295], 'medium duration_ms': [180494.499204635, 277791.1256366565], 'low duration_ms': [8586, 180494.499204635], 'high danceability': [0.6382691939772152, 0.985], 'medium danceability': [0.48605865656399494, 0.6382691939772152], 'low danceability': [0.0, 0.48605865656399494], 'high energy': [0.7449811392830035, 1.0], 'medium energy': [0.5239287468845211, 0.7449811392830035], 'low energy': [0.0, 0.5239287468845211], 'high loudness': [-6.2500393083862695, 4.532], 'medium loudness': [-10.748091817015506, -6.2500393083862695], 'low loudness': [-49.531, -10.748091817015506], 'high speechiness': [0.13623204572910647, 0.965], 'medium speechiness': [0.03864950564260645, 0.13623204572910647], 'low speechiness': [0.0, 0.03864950564260645], 'high acousticness': [0.4740026692392782, 0.996], 'medium acoustic

In [9]:
# text = "I want songs that are sad"
# classes_verbalized = list(labels.keys())
# zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here
# output = zeroshot_classifier(text, classes_verbalized, multi_label=True)
# print(output)
# #MoritzLaurer/ModernBERT-large-zeroshot-v2.0
# #MoritzLaurer/deberta-v3-large-zeroshot-v2.0

In [10]:
def generate_zsc_output(prompt):
    classes_verbalized = list(labels.keys())
    zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
    output = zeroshot_classifier(prompt, classes_verbalized, multi_label=True)
    return output

In [11]:
features

['popularity',
 'duration_ms',
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']

In [25]:
test_playlist = [df.iloc[random.randint(0, len(df))]["track_id"] for i in range(100)]

#use_labels = [output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.95]
#METHOD 1: normal --> using top three labels. if a song fits ANY of the three criteria, it is added
#TODO: change this to work with new file format --> genre having multiple values

test_input = "I want the songs that are sad but fast"
output = generate_zsc_output(test_input)
use_labels = [output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.95]
print(len(use_labels))

final_playlist = []
for label in use_labels:
    print(label)
    label_val = labels[label]
    col = None 
    if label_val is None:
        #if the label is a genre
        col = "track_genre"
    else:
        #if label is any other category
        ind = list(labels.keys()).index(label)
        col = features[ind // 3]
    for song_id in test_playlist:
        row = df[df["track_id"] == song_id]
        if (col == "track_genre"):
            if row[col].iloc[0] == label:
                if song_id not in final_playlist:
                    final_playlist.append(song_id)
        else:
            min = label_val[0]
            max = label_val[1]
            if row[col].iloc[0] <= max and row[col].iloc[0] >= min:
                if song_id not in final_playlist:
                    final_playlist.append(song_id)
final_playlist
len(final_playlist)

Device set to use mps:0


2
low speechiness
low duration_ms


52

In [15]:
test_playlist = [df.iloc[random.randint(0, len(df))]["track_id"] for i in range(100)]

#use_labels = [output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.95]
#METHOD 2: only songs that fit ALL criteria
test_input = "I want the songs that are sad but fast"
output = generate_zsc_output(test_input)
use_labels = [output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.97]
print(use_labels)
final_playlist = []
for song_id in test_playlist:
    row = df[df["track_id"] == song_id]
    satisfies_all = True
    for label in use_labels:
        label_val = labels[label]
        if label_val is None:
            if row["track_genre"].iloc[0] != label:
                satisfies_all = False
                break
        else:
            ind = list(labels.keys()).index(label)
            col = features[ind // 3]
            min = label_val[0]
            max = label_val[1]
            if not (row[col].iloc[0] <= max and row[col].iloc[0] >= min):
                satisfies_all = False
                break
    if satisfies_all:
        final_playlist.append(song_id)
            
final_playlist
len(final_playlist)

Device set to use mps:0


['low speechiness']


33

In [17]:
mod = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
def generate_subplaylist(playlist, prompt):
    final_playlist = []
    print(prompt)
    zsc = pipeline("zero-shot-classification", model=mod)
    output = zsc(prompt, classes_verbalized, multi_label=True)

    use_labels = [output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.8]

    for label in use_labels:
        label_type = type(labels[label])
        #for song in test_playlist:
        print(label_type)   

generate_subplaylist(test_playlist, test_input)


I want the songs that are sad but fast


Device set to use mps:0


NameError: name 'classes_verbalized' is not defined

In [23]:
test_input = "I want the songs that are happy"
output = generate_zsc_output(test_input)
print(output)
print([output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.90])

Device set to use mps:0


{'sequence': 'I want the songs that are happy', 'labels': ['low speechiness', 'high valence', 'low tempo', 'singer-songwriter', 'low duration_ms', 'pop', 'songwriter', 'low loudness', 'low instrumentalness', 'low popularity', 'happy', 'medium loudness', 'medium duration_ms', 'folk', 'medium popularity', 'medium speechiness', 'medium liveness', 'idm', 'medium tempo', 'medium instrumentalness', 'chill', 'pop-film', 'medium acousticness', 'low acousticness', 'high duration_ms', 'cantopop', 'high popularity', 'low energy', 'medium danceability', 'mandopop', 'medium energy', 'edm', 'low liveness', 'r-n-b', 'high energy', 'high liveness', 'children', 'indie-pop', 'rock-n-roll', 'kids', 'low danceability', 'comedy', 'show-tunes', 'mpb', 'medium valence', 'alternative', 'groove', 'disco', 'synth-pop', 'power-pop', 'high tempo', 'indie', 'rock', 'k-pop', 'club', 'reggae', 'hip-hop', 'high loudness', 'soul', 'sertanejo', 'reggaeton', 'disney', 'pagode', 'high acousticness', 'high danceability', 

In [21]:
test_input = "I want the songs that are happy and fast"
output = generate_zsc_output(test_input)
print(output)
print([output["labels"][x] for x in range(len(output["labels"])) if output["scores"][x] > 0.93])

Device set to use mps:0


{'sequence': 'I want the songs that are happy and fast', 'labels': ['low speechiness', 'low duration_ms', 'high tempo', 'pop', 'high energy', 'high valence', 'happy', 'singer-songwriter', 'low acousticness', 'idm', 'high duration_ms', 'rock-n-roll', 'rock', 'low popularity', 'power-pop', 'high danceability', 'medium popularity', 'pop-film', 'songwriter', 'medium loudness', 'medium speechiness', 'low loudness', 'cantopop', 'low instrumentalness', 'high liveness', 'medium liveness', 'high popularity', 'r-n-b', 'mandopop', 'edm', 'hard-rock', 'children', 'kids', 'indie-pop', 'hip-hop', 'club', 'medium instrumentalness', 'high loudness', 'medium acousticness', 'synth-pop', 'disco', 'folk', 'comedy', 'hardstyle', 'party', 'mpb', 'low liveness', 'honky-tonk', 'medium danceability', 'medium valence', 'funk', 'show-tunes', 'k-pop', 'alt-rock', 'high instrumentalness', 'electro', 'medium duration_ms', 'groove', 'electronic', 'alternative', 'rockabilly', 'pagode', 'guitar', 'drum-and-bass', 'tec